2022/10  --  Felix Klotzsche

This is a pretty dirty NB that I used to manually check problematic raw files and did some sanity checks. Probably not helpful in the current state but might have some useful snippets, so I keep it. 



In [1]:
import os
from pathlib import Path

import mne
import numpy as np
import pandas as pd

from vr2f.staticinfo import PATHS

In [4]:
paths = PATHS()
sub_problematic = ["VR2FEM_S03"]
sub_path = paths.DATA_SUBJECTS
sub_list_str = os.listdir(sub_path)
mega_df = pd.DataFrame(columns=["id"])
ev_coll = {}
for sub_id in sub_list_str:
    if sub_id in sub_problematic:
        continue
    p_data = Path(sub_path, sub_id, "MainTask", "EEG")
    fname = Path(p_data, sub_id + ".vhdr")
    raw = mne.io.read_raw_brainvision(fname, preload=False, verbose=False)
    evs, ev_dict = mne.events_from_annotations(raw, verbose=False)
    ev_coll[sub_id] = evs
    df_evs = pd.DataFrame(evs, columns=["time", "dur", "id"])
    df_evs = df_evs.groupby("id").size().reset_index(name=sub_id)
    if np.any(df_evs.loc[(df_evs["id"] > 110) & (df_evs["id"] < 235), sub_id] != 31):  # noqa: PLR2004
        mega_df = mega_df.merge(df_evs, how="outer")

In [5]:
paths = PATHS()
sub_id = "VR2FEM_S19"

eeg_data = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "EEG")
fname1 = Path(eeg_data, sub_id + ".vhdr")
raw1 = mne.io.read_raw_brainvision(fname1, preload=True, verbose=False)

evs, ev_dict = mne.events_from_annotations(raw1, verbose=False)
df_ev = pd.DataFrame(evs, columns=["time", "dur", "id"])
df_ev = df_ev.groupby("id").size().reset_index(name=sub_id)
df_ev

,id,VR2FEM_S19
0,11,743
1,13,743
2,14,741
3,15,3
4,16,744
5,111,31
6,112,31
7,113,31
8,114,31
9,121,31


In [17]:
sub_id = "VR2FEM_S23"
eeg_data = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "EEG")
fname1 = Path(eeg_data, sub_id + ".vhdr")
raw1 = mne.io.read_raw_brainvision(fname1, preload=True, verbose=False)
# delete first 2 events & we lost some events due to bad connection:
n_ev2skip = 2
counter = 0
idx_del = []
for idx, a in enumerate(raw1.annotations):
    sdesc = a["description"]
    if "Stimulus" in sdesc and counter < n_ev2skip:
        idx_del.append(idx)
        if sdesc == "Stimulus/S 16":  # trial end
            counter += 1
raw1.annotations.delete(idx_del)

In [56]:
ev23, ed = mne.events_from_annotations(raw1, verbose=False)
idx_stims = (ev23[:, 2] > 100) & (ev23[:, 2] < 400)  # noqa: PLR2004
stims23 = ev23[idx_stims, 2]

ftres = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "Unity", "S001", "trial_results.csv")
tres23 = pd.read_csv(ftres)
start_prob = np.where(np.array(tres23["marker"])[: len(stims23)] != stims23)[0][0]

compare = np.array([np.array(tres23["marker"])[: len(stims23)], stims23])

stims23_1 = np.insert(stims23, start_prob, 112)
compare_1 = np.array([np.array(tres23["marker"])[: len(stims23_1)], stims23_1])
start_prob_1 = np.where(compare_1[0, :] != compare_1[1, :])[0][0]

stims23_2 = np.insert(stims23_1, start_prob_1, [122, 131])
np.all(stims23_2 == tres23["marker"])
print(start_prob)
print(start_prob_1)

208
610


In [4]:
# sub S32
paths = PATHS()
sub_id = "VR2FEM_S32"
eeg_data = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "EEG")
fname1 = Path(eeg_data, sub_id + ".vhdr")
raw1 = mne.io.read_raw_brainvision(fname1, preload=True, verbose=False)
ev32, ed = mne.events_from_annotations(raw1, verbose=False)
idx_stims = (ev32[:, 2] > 100) & (ev32[:, 2] < 400)  # noqa: PLR2004
stims32 = ev32[idx_stims, 2]

ftres32 = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "Unity", "S001", "trial_results.csv")
tres32 = pd.read_csv(ftres32)


compare = np.array([np.array(tres32["marker"])[: len(stims32)], stims32])
start_prob = np.where(compare[0, :] != compare[1, :])[0][0]
print(start_prob)
aa = compare.T

In [7]:
# sub S06
paths = PATHS()
sub_id = "VR2FEM_S06"
eeg_data = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "EEG")
fname1 = Path(eeg_data, sub_id + ".vhdr")
raw1 = mne.io.read_raw_brainvision(fname1, preload=True, verbose=False)
ev06, ed = mne.events_from_annotations(raw1, verbose=False)
idx_stims = (ev06[:, 2] > 100) & (ev06[:, 2] < 400)  # noqa: PLR2004
stims06 = ev06[idx_stims, 2]


ftres06 = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "Unity", "S001", "other", "markerLog.csv")
tres06 = pd.read_csv(ftres06)

ha = tres06[tres06.melt().str.contains("Stimulus Onset").any(level=0)]
mrkrs = ha.annotation.str.extract(r"(\d+)")
mrkrs.columns = ["marker"]
mrkrs_i = pd.to_numeric(mrkrs.marker)

/tmp/ipykernel_68852/599434576.py:15: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.any(level=1) should use df.groupby(level=1).any()
  ha = tres06[tres06.stack().str.contains('Stimulus Onset').any(level=0)]


,0
1,221
6,123
11,122
16,113
21,223
...,...
3696,131
3701,231
3706,223
3711,132


In [19]:
compare = np.array([np.array(mrkrs_i)[: len(stims06)], stims06])
start_prob = np.where(compare[0, :] != compare[1, :])[0][0]
print(start_prob)
aa = compare.T

610


In [10]:
# sub S03
paths = PATHS()
sub_id = "VR2FEM_S03"
eeg_data = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "EEG")
fname1 = Path(eeg_data, sub_id + ".vhdr")
raw1 = mne.io.read_raw_brainvision(fname1, preload=True, verbose=False)
ev03, ed = mne.events_from_annotations(raw1, verbose=False)
idx_stims = (ev03[:, 2] > 100) & (ev03[:, 2] < 400)  # noqa: PLR2004
stims03 = ev03[idx_stims, 2]
# cut off last 4 stims
stims03 = stims03[:-4]


ftres03 = Path(paths.DATA_SUBJECTS, sub_id, "MainTask", "Unity", "S001", "other", "markerLog.csv")
tres03 = pd.read_csv(ftres03)

ha = tres03[tres03.melt().str.contains("Stimulus Onset").any(level=0)]
mrkrs = ha.annotation.str.extract(r"(\d+)")
mrkrs.columns = ["marker"]
mrkrs_i = pd.to_numeric(mrkrs.marker)

/tmp/ipykernel_35055/1556373055.py:18: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.any(level=1) should use df.groupby(level=1).any()
  ha = tres03[tres03.stack().str.contains('Stimulus Onset').any(level=0)]


In [ ]:


sub_id = "VR2FEM_S19"
sub_evs = ev_coll[sub_id]

# check whether every stim onset is followed by a stim offset event:
idx_stimon = np.where((sub_evs[:,2]>110) & (sub_evs[:,2]<235))  # noqa: PLR2004
if (np.any(sub_evs[idx_stimon[0] + 1, 2] != 13)):  # noqa: PLR2004
    raise ValueError(f"{sub_id}:    NOt each stim onset followed by offset event.")

stim_onsets = sub_evs[idx_stimon, :]
stim_offsets = sub_evs[(sub_evs[:,2]==13), :]  # noqa: PLR2004
if (len(stim_onsets) != len(stim_offsets)):
    raise ValueError(f"{sub_id}:    Unequal number of stim on- and offsets.")
diffs = (stim_offsets[:,0] - stim_onsets[:,0])  # [:-1,0])



idx = np.where(sub_evs[:,0] == stim_onsets[631,0])
sub_evs[idx[0][0]-10:idx[0][0]+10]
diffs[np.where(diffs > 510)]  # noqa: PLR2004
diffs

In [57]:
fname = Path(p_data, sub_id + ".vhdr")
raw = mne.io.read_raw_brainvision(fname, preload=False)

Extracting parameters from ../Data/Subjects/VR2FEM_S28/MainTask/EEG/VR2FEM_S28.vhdr...
Setting channel info structure...


In [62]:
np.sum(evs[0][:, 2] == 211)  # noqa: PLR2004
df_ev = pd.DataFrame(evs[0], columns=["time", "dur", "id"])
df_ev.groupby("id").size().reset_index(name="counts")

,id,counts
0,11,744
1,13,744
2,14,742
3,15,2
4,16,744
5,111,31
6,112,31
7,113,31
8,114,31
9,121,31
